In [ ]:
import pandas as pd
import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY") 

In [ ]:
#ChatGPT Example of a scouting report
scouting_report = """
John Smith is known for his calm demeanor and ability to rally teammates in high-pressure moments. 
He has shown strong communication skills and often takes on a mentoring role with younger players. 
Coaches have praised his resilience and willingness to lead by example.
"""

In [ ]:
#Define Prompts and pull scores
def get_qualitative_scores(report_text, player_name):
    prompt = f"""
    Based on the following scouting report for {player_name}, give a score from 1 to 10 for each of the following traits:
    - Leadership
    - Clutch Performance
    - Injury Risk (1 = very risky, 10 = very durable)

    Report:
    \"\"\"{report_text}\"\"\"

    Return the response in this format:
    Leadership: [score]
    Clutch Performance: [score]
    Injury Risk: [score]
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a baseball analyst."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    return response.choices[0].message['content']

In [ ]:
#Response Dictionary
import re

def parse_scores(response_text):
    scores = {}
    matches = re.findall(r"(\w[\w\s]+):\s*(\d+)", response_text)
    for trait, score in matches:
        scores[trait.strip()] = int(score)
    return scores

In [ ]:
#Create player dataframe to merge with quantitative data
players = [
    {
        "name": "John Smith",
        "report": """John Smith is known for his calm demeanor and ability to rally teammates..."""
    },
    {
        "name": "Derek Ray",
        "report": """Derek Ray is highly athletic but has a history of minor injuries..."""
    }
]

# Loop through each player
for player in players:
    gpt_response = get_qualitative_scores(player['report'], player['name'])
    traits = parse_scores(gpt_response)
    player.update(traits)

# Convert to DataFrame to merge with numerical data
qualitative_df = pd.DataFrame(players)
print(qualitative_df[['name', 'Leadership', 'Clutch Performance', 'Injury Risk']])
